<a href="https://colab.research.google.com/github/EduChicoRosa/Arquivos-Publicos/blob/main/An%C3%A1lise_de_paradas_para_limpeza_de_tocha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter

# ==============================================================================
# 0. CONFIGURAÇÕES IMPORTANTES
# ==============================================================================
# Nomes das colunas ajustados conforme o seu código funcional
TIMESTAMP_COLUMN = 'TimeStamp'
VALUE_COLUMN = 'Value'
MACHINE_COLUMN = 'StationSAP'
SIGNAL_COLUMN = 'VariableName'


# ==============================================================================
# 1. CARREGAMENTO DOS DADOS
# ==============================================================================

print("Por favor, faça o upload do seu arquivo CSV de sinais.")
uploaded = files.upload()
file_name = next(iter(uploaded))

try:
    # Tentando com ',' como separador
    df = pd.read_csv(file_name, sep=',')
    print(f"\nArquivo '{file_name}' carregado com sucesso!")
except Exception as e:
    print(f"\nErro ao ler o arquivo com ','. Tentando com ';'. Erro: {e}")
    try:
        df = pd.read_csv(file_name, sep=';')
        print("Arquivo lido com sucesso usando ';' como separador.")
    except Exception as e2:
        print(f"Não foi possível ler o arquivo. Erro: {e2}")
        exit() # Encerra a execução se o arquivo não puder ser lido


# ==============================================================================
# 2. PREPARAÇÃO DOS DADOS
# ==============================================================================

# Converte 'StationSAP' para numérico, tratando erros
df[MACHINE_COLUMN] = pd.to_numeric(df[MACHINE_COLUMN], errors='coerce')
df.dropna(subset=[MACHINE_COLUMN], inplace=True) # Remove linhas onde StationSAP não é numérico

# Converte a coluna de timestamp para o formato datetime
try:
    df[TIMESTAMP_COLUMN] = pd.to_datetime(df[TIMESTAMP_COLUMN])

    ### CORREÇÃO DE FUSO HORÁRIO 1/2 ###
    # Assume que os timestamps no arquivo estão no horário local (Brasil).
    # Localizamos esses timestamps para o fuso correto para evitar erros de comparação.
    df[TIMESTAMP_COLUMN] = df[TIMESTAMP_COLUMN].dt.tz_localize('America/Sao_Paulo', ambiguous='infer')
    print("\nTimestamps localizados para o fuso horário de São Paulo (UTC-3).")

except KeyError:
    print(f"ERRO: A coluna de timestamp '{TIMESTAMP_COLUMN}' não foi encontrada.")
    exit()
except Exception as e:
    print(f"ERRO ao converter ou localizar a coluna de timestamp: {e}")
    exit()

# Garante que a coluna de valor seja numérica para os cálculos
try:
    df[VALUE_COLUMN] = pd.to_numeric(df[VALUE_COLUMN], errors='coerce')
    df.dropna(subset=[VALUE_COLUMN], inplace=True) # Remove linhas onde o valor não é numérico
except KeyError:
    print(f"ERRO: A coluna de valor '{VALUE_COLUMN}' não foi encontrada.")
    exit()


# ==============================================================================
# 2.1. FILTRO DE PERÍODO (NOVA FUNCIONALIDADE)
# ==============================================================================
print("\n--- Selecione o Período da Análise ---")
print("1: Última hora")
print("2: Últimas 24 horas")
print("3: Última semana (7 dias)")
print("4: Últimos 30 dias")

period_choice = input("Digite o número da opção desejada (1-4): ")


### CORREÇÃO DE FUSO HORÁRIO 2/2 ###
# Pega o horário atual já localizado para o fuso correto para garantir
# que o filtro de tempo seja calculado corretamente.
now = pd.Timestamp.now(tz='America/Sao_Paulo')
start_date = None

if period_choice == '1':
    start_date = now - pd.Timedelta(hours=1)
elif period_choice == '2':
    start_date = now - pd.Timedelta(hours=24)
elif period_choice == '3':
    start_date = now - pd.Timedelta(days=7)
elif period_choice == '4':
    start_date = now - pd.Timedelta(days=30)
else:
    print("Opção inválida. A análise será executada com todos os dados.")

if start_date:
    df = df[df[TIMESTAMP_COLUMN] >= start_date].copy()
    print(f"\nAnálise será executada para dados a partir de: {start_date.strftime('%Y-%m-%d %H:%M:%S')}")

if df.empty:
    print("\nNenhum dado encontrado para o período selecionado. A análise será encerrada.")
    exit()

# Ordena os dados após o filtro
df.sort_values(by=[MACHINE_COLUMN, SIGNAL_COLUMN, TIMESTAMP_COLUMN], inplace=True)


# ==============================================================================
# 3. CÁLCULO DE INTERVALOS DE ATIVAÇÃO DOS SINAIS
# ==============================================================================

print("\nCalculando os intervalos em que cada sinal esteve ativo...")

df['prev_value'] = df.groupby([MACHINE_COLUMN, SIGNAL_COLUMN])[VALUE_COLUMN].shift(1)
start_events = df[(df[VALUE_COLUMN] == 1) & (df['prev_value'] != 1)]
end_events = df[(df[VALUE_COLUMN] == 0) & (df['prev_value'] == 1)]

starts = start_events[[MACHINE_COLUMN, SIGNAL_COLUMN, TIMESTAMP_COLUMN]].rename(columns={TIMESTAMP_COLUMN: 'start_time'})
ends = end_events[[MACHINE_COLUMN, SIGNAL_COLUMN, TIMESTAMP_COLUMN]].rename(columns={TIMESTAMP_COLUMN: 'end_time'})

intervals = pd.merge_asof(
    starts.sort_values('start_time'),
    ends.sort_values('end_time'),
    left_on='start_time',
    right_on='end_time',
    by=[MACHINE_COLUMN, SIGNAL_COLUMN],
    direction='forward'
)

intervals.dropna(subset=['end_time'], inplace=True)
intervals['duration_seconds'] = (intervals['end_time'] - intervals['start_time']).dt.total_seconds()
print("Cálculo de intervalos concluído.")


# ==============================================================================
# 4. ANÁLISE 1: TEMPO ATIVO DE 'LimpTochaProx' E 'LimpTochaAct'
# ==============================================================================

print("\n--- Análise 1: Tempo total de atividade dos sinais 'LimpTocha' ---")
limp_signals = intervals[intervals[SIGNAL_COLUMN].isin(['LimpTochaProx', 'LimpTochaAct'])]
active_time_summary = limp_signals.groupby([MACHINE_COLUMN, SIGNAL_COLUMN])['duration_seconds'].sum().reset_index()
active_time_summary['duration_minutes'] = active_time_summary['duration_seconds'] / 60

if active_time_summary.empty:
    print("Nenhuma atividade encontrada para os sinais 'LimpTochaProx' ou 'LimpTochaAct' no período.")
else:
    print(active_time_summary[[MACHINE_COLUMN, SIGNAL_COLUMN, 'duration_minutes']])
    active_time_summary.to_csv('tempo_ativo_limp_tocha.csv', index=False)
    print("\nResultado salvo em 'tempo_ativo_limp_tocha.csv'")


# ==============================================================================
# 5. ANÁLISE 2: SOBREPOSIÇÃO DE 'LimpTochaAct' E 'Passo6mesaX'
# ==============================================================================

print("\n--- Análise 2: Tempo de parada (sobreposição de LimpTochaAct e Passo6mesaX) ---")
limp_act_intervals = intervals[intervals[SIGNAL_COLUMN] == 'LimpTochaAct'].copy()
mesa_intervals = intervals[intervals[SIGNAL_COLUMN].str.startswith('Passo6mesa', na=False)].copy()

downtimes_df = pd.DataFrame() # DataFrame para armazenar todas as paradas

if limp_act_intervals.empty or mesa_intervals.empty:
    print("Não foram encontrados os sinais necessários ('LimpTochaAct' e/ou 'Passo6mesaX') para a análise de sobreposição no período.")
else:
    merged_intervals = pd.merge(
        limp_act_intervals,
        mesa_intervals,
        on=MACHINE_COLUMN,
        suffixes=('_limp', '_mesa')
    )

    overlapping = merged_intervals[
        (merged_intervals['start_time_limp'] < merged_intervals['end_time_mesa']) &
        (merged_intervals['start_time_mesa'] < merged_intervals['end_time_limp'])
    ].copy()

    if not overlapping.empty:
        overlapping['overlap_start'] = np.maximum(overlapping['start_time_limp'], overlapping['start_time_mesa'])
        overlapping['overlap_end'] = np.minimum(overlapping['end_time_limp'], overlapping['end_time_mesa'])
        overlapping['overlap_seconds'] = (overlapping['overlap_end'] - overlapping['overlap_start']).dt.total_seconds()
        # Filtra paradas com duração maior que zero
        downtimes_df = overlapping[overlapping['overlap_seconds'] > 0].copy()

        stoppage_summary = downtimes_df.groupby(MACHINE_COLUMN)['overlap_seconds'].sum().reset_index()
        stoppage_summary.rename(columns={'overlap_seconds': 'total_stoppage_seconds'}, inplace=True)
        stoppage_summary['total_stoppage_minutes'] = stoppage_summary['total_stoppage_seconds'] / 60

        if stoppage_summary.empty:
            print("Nenhuma sobreposição encontrada entre 'LimpTochaAct' e sinais 'Passo6mesaX' no período.")
        else:
            print(stoppage_summary[[MACHINE_COLUMN, 'total_stoppage_minutes']])
            stoppage_summary.to_csv('tempo_parada_por_sobreposicao.csv', index=False)
            print("\nResultado salvo em 'tempo_parada_por_sobreposicao.csv'")

# ==============================================================================
# 6. ANÁLISE 3: MAIORES PARADAS E PARETO DE CAUSAS (NOVA FUNCIONALIDADE)
# ==============================================================================

print("\n--- Análise 3: Maiores Paradas e Pareto de Causas ---")

if downtimes_df.empty:
    print("Nenhuma parada identificada para a análise de Pareto.")
else:
    # --- Tabela: 3 Maiores Paradas Individuais por Máquina ---
    print("\n-- Top 3 Maiores Paradas Individuais por Máquina --")
    top_3_downtimes = downtimes_df.sort_values('overlap_seconds', ascending=False).groupby(MACHINE_COLUMN).head(3)

    # Formata a tabela para melhor visualização
    top_3_downtimes_report = top_3_downtimes[[MACHINE_COLUMN, 'overlap_start', 'overlap_end', 'overlap_seconds', 'VariableName_mesa']].copy()
    top_3_downtimes_report['Dia'] = top_3_downtimes_report['overlap_start'].dt.strftime('%Y-%m-%d')
    top_3_downtimes_report['Hora_Inicio'] = top_3_downtimes_report['overlap_start'].dt.strftime('%H:%M:%S')
    top_3_downtimes_report['Hora_Fim'] = top_3_downtimes_report['overlap_end'].dt.strftime('%H:%M:%S')
    top_3_downtimes_report['Duracao_Minutos'] = top_3_downtimes_report['overlap_seconds'] / 60

    print(top_3_downtimes_report[[MACHINE_COLUMN, 'Dia', 'Hora_Inicio', 'Hora_Fim', 'Duracao_Minutos', 'VariableName_mesa']].rename(columns={MACHINE_COLUMN: 'Machine', 'VariableName_mesa': 'Causa'})) # Fixed column name
    top_3_downtimes_report.to_csv('top_3_maiores_paradas.csv', index=False)
    print("\nRelatório das 3 maiores paradas salvo em 'top_3_maiores_paradas.csv'")

    # --- Gráfico de Pareto por Máquina ---
    print("\nGerando gráficos de Pareto por máquina...")
    downtime_causes = downtimes_df.groupby([MACHINE_COLUMN, 'VariableName_mesa'])['overlap_seconds'].sum().reset_index()

    for machine in downtime_causes[MACHINE_COLUMN].unique():
        machine_causes = downtime_causes[downtime_causes[MACHINE_COLUMN] == machine].sort_values('overlap_seconds', ascending=False).copy()
        machine_causes['cumulative_perc'] = 100 * machine_causes['overlap_seconds'].cumsum() / machine_causes['overlap_seconds'].sum()

        fig, ax = plt.subplots(figsize=(12, 7))

        # Gráfico de Barras
        ax.bar(machine_causes['VariableName_mesa'], machine_causes['overlap_seconds'] / 60, color="C0")
        ax.set_title(f'Pareto de Causas de Parada - Máquina: {int(machine)}', fontsize=16)
        ax.set_xlabel('Causa da Parada (Sinal Passo6mesaX)', fontsize=12)
        ax.set_ylabel('Duração Total da Parada (Minutos)', fontsize=12)
        plt.xticks(rotation=45, ha='right')

        # Eixo secundário para a porcentagem acumulada
        ax2 = ax.twinx()
        ax2.plot(machine_causes['VariableName_mesa'], machine_causes['cumulative_perc'], color="C1", marker="o", ms=5)
        ax2.yaxis.set_major_formatter(PercentFormatter())
        ax2.set_ylabel('Porcentagem Acumulada', fontsize=12)

        plt.tight_layout()
        pareto_filename = f'pareto_maquina_{int(machine)}.png'
        plt.savefig(pareto_filename)
        print(f"- Gráfico '{pareto_filename}' salvo.")
        plt.close(fig)

print("\nAnálise concluída!")

Por favor, faça o upload do seu arquivo CSV de sinais.


Saving ProcessData_BRMatriz.csv to ProcessData_BRMatriz.csv


/tmp/ipython-input-2746396868.py:28: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, sep=',')



Arquivo 'ProcessData_BRMatriz.csv' carregado com sucesso!

Timestamps localizados para o fuso horário de São Paulo (UTC-3).

--- Selecione o Período da Análise ---
1: Última hora
2: Últimas 24 horas
3: Última semana (7 dias)
4: Últimos 30 dias
Digite o número da opção desejada (1-4): 4

Análise será executada para dados a partir de: 2025-10-25 14:23:50

Calculando os intervalos em que cada sinal esteve ativo...
Cálculo de intervalos concluído.

--- Análise 1: Tempo total de atividade dos sinais 'LimpTocha' ---
   StationSAP   VariableName  duration_minutes
0     13306.0   LimpTochaAct        605.266667
1     13306.0  LimpTochaProx       2701.433333
2     13307.0   LimpTochaAct        375.316667
3     13307.0  LimpTochaProx       4784.100000
4     13308.0   LimpTochaAct        284.416667
5     13308.0  LimpTochaProx       2154.400000

Resultado salvo em 'tempo_ativo_limp_tocha.csv'

--- Análise 2: Tempo de parada (sobreposição de LimpTochaAct e Passo6mesaX) ---
   StationSAP  total_sto